In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import pandas as pd
from sklearn import dummy, metrics, linear_model, model_selection, neighbors
from sklearn.model_selection import train_test_split

# Introdução ao Aprendizado de Máquina

## Trabalhando com dados

### Carregando os datasets

Nós iremos ver duas vertentes do mesmo dataset, com propósitos diferentes. `Weather` é um dataset educacional (composto por dados fictícios) amplamente usado para ensinar conceitos de aprendizado de máquina. Ele contém 14 instâncias, 4 features e é considerado um dataset de classificação binária.

In [ ]:
weather_nominal = pd.read_csv('./weather.nominal.csv')
weather_nominal

weather_numeric = pd.read_csv('./weather.csv')
weather_numeric

Aqui vamos conferir os possíveis valores únicos em colunas categóricas:

In [ ]:
print('Outlook:', weather_nominal.outlook.unique())
print('Temperature:', weather_nominal.temperature.unique())
print('Humidity:', weather_nominal.humidity.unique())
print('Windy:', weather_nominal.windy.unique())
print('Play:', weather_nominal.play.unique())

Mostrando dataset com `head` (5 primeiros valores) e `tail` (5 últimos valores), bom para datasets extensos.

In [ ]:
weather_nominal.head()
weather_numeric.tail()

### Transformando os dados categóricos em numéricos

Para usar os datasets, nós devemos transformar os dados categóricos em valores numéricos, já que tanto nosso método de classificação quanto o de regressão trabalham apenas com números. Iremos usar uma técnica de encoding básica, mapeando os valores categóricos para números.

Uma forma mais "profissional" de fazer a transformação de dados é o `One-hot encoding`, mas não será tratado aqui.

In [ ]:
# Mudando o tipo das colunas categóricas
weather_numeric['outlook'] = weather_numeric['outlook'].astype('category')
weather_numeric['windy'] = weather_numeric['windy'].astype('category')
weather_numeric['play'] = weather_numeric['play'].astype('category')

# Aplicando enconding às colunas selecionadas
cat_columns = weather_numeric.select_dtypes(['category']).columns
weather_numeric[cat_columns] = weather_numeric[cat_columns].apply(lambda x: x.cat.codes)
weather_numeric.head()

### Dividindo datasets em treino e teste

In [ ]:
X_nominal_train, X_nominal_test, y_nominal_train, y_nominal_test = train_test_split(
    weather_nominal.iloc[:, :-1], 
    weather_nominal.iloc[:, -1], 
    test_size=0.3
)

X_nominal_train
X_nominal_test
y_nominal_train
y_nominal_test

In [ ]:
X_numeric_train, X_numeric_test, y_numeric_train, y_numeric_test = train_test_split(
    weather_numeric.iloc[:, :-1], 
    weather_numeric.iloc[:, -1], 
    test_size=0.3
)

X_numeric_train
X_numeric_test
y_numeric_train
y_numeric_test

## Modelos

### Classificação

Vamos usar o KNN para classificar as instâncias do `Weather`. Como funciona o KNN? Imagine que suas instâncias estão distribuídas pelo espaço (então você tem a ideia de pontos e o conceito de distâncias) e que instâncias próximas são semelhantes. Logo, você pode "adivinhar" a classe de uma instância olhando para seus vizinhos.  

In [ ]:
classification = neighbors.KNeighborsClassifier(n_neighbors=3)
classification.fit(X_numeric_train, y_numeric_train)

In [ ]:
class_predict = classification.predict(X_numeric_test)
class_predict

### Regressão

Vamos agora usar a Regressão Linear para fazer a regressão das instâncias do `Weather`. A ideia da regressão linear é explicar seus dados como uma função `y = a * x + b + e`, onde `a` e `b` são coeficientes e `e` é um termo de erro, ou seja, uma medida que mostra que existem outros fatores não explicados pelo modelo.

In [ ]:
regression = linear_model.LinearRegression()
regression.fit(X_numeric_train, y_numeric_train)

In [ ]:
regr_predict = regression.predict(X_numeric_test)
regr_predict

### Avaliação de resultados

Existem várias formas de medir quão "bom" é um modelo (apesar de isso ser subjetivo), as métricas são boas formas. Elas testam resultados gerados pelos modelos com valores reais, e dão uma ideia de quão preciso é o seu modelo na classificação (ou quão bem sua curva explica os dados na regressão).

In [ ]:
print('KNN:')
print(metrics.classification_report(y_numeric_test, class_predict))

print('Regressão:')
print(metrics.mean_squared_error(y_numeric_test, regr_predict))

### Cross validation

A ideia é evitar o overfitting fazendo passos intermediários de validação com uma parte do dataset de treino. Por exemplo, uma validação cruzada de 10 folds treina o modelo com 9 pedaços e valida os resultados obtidos com o pedaço restante, e repete esse processo mais 9 vezes.

In [ ]:
print('KNN:')
print('validação cruzada:', model_selection.cross_validate(
    classification, 
    X_numeric_train, 
    y_numeric_train,
    cv=2
))

print('Regressão:')
print('validação cruzada:', model_selection.cross_validate(
    regression, 
    X_numeric_train, 
    y_numeric_train,
    cv=2
))

## Extras

### Baseline classifier

É muito difícil avaliar o resultado de um modelo (descobrir se o valor resultante das métricas é bom ou não), mas existe uma forma de descobrir qual seria o pior valor possível. A ideia é usar um classificador "dummy", isto é, uma estratégia ingênua de classificação. Neste caso apresentado abaixo, o dummy checa qual a classe dominante e classifica as instâncias de teste com aquela mesma classe.

In [ ]:
classification = dummy.DummyClassifier(strategy='most_frequent')
classification.fit(X_nominal_train, y_nominal_train)

In [ ]:
pred = classification.predict(X_nominal_test)

metrics.accuracy_score(y_nominal_test, pred)